In [1]:
!mkdir banners

In [12]:
!pip install psd-tools

In [13]:
import os
import pandas as pd
from psd_tools import PSDImage

In [14]:
folder_path = "/content/banners"
all_data = []
kv_data = {}  # Info de los Key Visuals (KV)

In [15]:
# Asignar tipo de elemento a cada capa
def assign_element_type(layer_name):
    name = layer_name.lower()
    if "fixed" in name or "forced" in name or "locked" in name:
        return "FIXED"
    elif "cta" in name or "button" in name:
        return "CTA"
    elif "product" in name or "sku" in name:
        return "PRODUCT"
    elif "logo" in name:
        return "LOGO"
    elif "legal" in name:
        return "LEGAL"
    elif "copy" in name or "text" in name or "sub heading" in name:
        return "TEXT"
    elif "background" in name:
        return "BACKGROUND"
    else:
        return "OTHER"

# Asignar prioridad según tipo de elemento
def assign_priority(element_type):
    if element_type == "FIXED":
        return 3
    elif element_type in ["CTA", "PRODUCT", "LOGO", "LEGAL", "TEXT"]:
        return 2
    elif element_type == "BACKGROUND":
        return -1
    else:
        return 0

# Extraer capas de un PSD
def extract_layers(psd, filename):
    elements = []
    canvas_width = psd.width
    canvas_height = psd.height

    for i, layer in enumerate(psd):
        if not layer.is_group() and layer.bbox is not None:
            left, top, right, bottom = layer.bbox
            width = right - left
            height = bottom - top
            element_type = assign_element_type(layer.name)
            priority = assign_priority(element_type)

            elements.append({
                'file': filename,
                'layer_name': layer.name,
                'type': element_type,
                'priority': priority,
                'x': left,
                'y': top,
                'width': width,
                'height': height,
                'z_index': i,
                'canvas_width': canvas_width,
                'canvas_height': canvas_height
            })
    return elements

# Obtener nombre base (sin dimensiones ni "KV")
def get_base_name(filename):
    parts = filename.replace(".psd", "").split("_")
    last_part = parts[-1].lower()
    if last_part == "kv" or "x" in last_part:
        parts = parts[:-1]
    return "_".join(parts)

# -------- PRIMERA PASADA: PROCESAR KVs -------- #
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".psd") and "kv" in filename.lower():
        filepath = os.path.join(folder_path, filename)
        psd = PSDImage.open(filepath)
        base_name = get_base_name(filename)

        elements = extract_layers(psd, filename)

        kv_data[base_name] = {
            'canvas_width': psd.width,
            'canvas_height': psd.height,
            'elements': elements
        }

        # Añadir también KV a all_data (sin comparación)
        for el in elements:
            all_data.append({
                **el,
                'kv_canvas_width': psd.width,
                'kv_canvas_height': psd.height,
                'kv_x': el['x'],  # KV comparado consigo mismo
                'kv_y': el['y'],
                'kv_width': el['width'],
                'kv_height': el['height'],
                'kv_z_index': el['z_index']
                # NOTA: aquí NO añadimos text_x, text_y… porque dijiste que no
            })

# -------- SEGUNDA PASADA: PROCESAR BANNERS -------- #
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".psd") and "kv" not in filename.lower():
        filepath = os.path.join(folder_path, filename)
        psd = PSDImage.open(filepath)
        base_name = get_base_name(filename)

        elements = extract_layers(psd, filename)

        kv_info = kv_data.get(base_name)
        kv_canvas_width = kv_canvas_height = None

        # Mapear por tipo para comparar fondo con fondo, producto con producto…
        kv_by_type = {}
        if kv_info:
            kv_canvas_width = kv_info['canvas_width']
            kv_canvas_height = kv_info['canvas_height']
            for el in kv_info['elements']:
                kv_by_type.setdefault(el['type'], []).append(el)

        for el in elements:
            # Buscar capa equivalente en KV por type
            kv_x = kv_y = kv_width = kv_height = kv_z = None
            if kv_by_type.get(el['type']):
                # tomar la primera del mismo tipo
                kv_match = kv_by_type[el['type']][0]
                kv_x = kv_match['x']
                kv_y = kv_match['y']
                kv_width = kv_match['width']
                kv_height = kv_match['height']
                kv_z = kv_match['z_index']

            all_data.append({
                **el,
                'kv_canvas_width': kv_canvas_width,
                'kv_canvas_height': kv_canvas_height,
                'kv_x': kv_x,
                'kv_y': kv_y,
                'kv_width': kv_width,
                'kv_height': kv_height,
                'kv_z_index': kv_z
            })

df = pd.DataFrame(all_data)
df.to_csv("banners_metadata.csv", index=False)
print(df.head())

                   file   layer_name        type  priority    x     y  width  \
0          Bubly_KV.psd   BACKGROUND  BACKGROUND        -1    0     0   1706   
1          Bubly_KV.psd  SUB HEADING        TEXT         2  381  1341    910   
2          Bubly_KV.psd         COPY        TEXT         2  172   219   1361   
3          Bubly_KV.psd         SKUs     PRODUCT         2  232   523   1273   
4  Gatorade_HBTW_KV.psd   BACKGROUND  BACKGROUND        -1    0     0   1920   

   height  z_index  canvas_width  canvas_height  kv_canvas_width  \
0    1706        0          1706           1706           1706.0   
1     146        1          1706           1706           1706.0   
2     162        2          1706           1706           1706.0   
3     570        3          1706           1706           1706.0   
4    1080        0          1920           1080           1920.0   

   kv_canvas_height   kv_x    kv_y  kv_width  kv_height  kv_z_index  
0            1706.0    0.0     0.0    17